<a href="https://colab.research.google.com/github/YOONSEOUNGHWAN/AI_TEST/blob/main/TEST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master


# torch
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm

#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-3.0.2-py3-none-any.whl (769 kB)
  Using cached tokenizers-0.8.1rc1-cp37-cp37m-manylinux1_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.12.1
    Uninstalling tokenizers-0.12.1:
      Successfully uninstalled tokenizers-0.12.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.20.1
    Uninstalling transformers-4.20.1:
      Successfully uninstalled transformers-4.20.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behav

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-blcckuoj
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-blcckuoj
  Using cached transformers-4.20.1-py3-none-any.whl (4.4 MB)
  Using cached tokenizers-0.12.1-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (6.6 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.8.1rc1
    Uninstalling tokenizers-0.8.1rc1:
      Successfully uninstalled tokenizers-0.8.1rc1
  Attempting uninstall: transformers
    Found existing installation: transformers 3.0.2
    Uninstalling transformers-3.0.2:
      Successfully uninstalled transformers-3.0.2


In [39]:
from google.colab import drive

In [40]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [99]:
#GPU 사용
# device = torch.device("cuda:0")

#BERT 모델, Vocabulary 불러오기 필수
bertmodel, vocab = get_pytorch_kobert_model()


# KoBERT에 입력될 데이터셋 정리
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))  

# 모델 정의
class BERTClassifier(nn.Module): ## 클래스를 상속
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=6,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

# Setting parameters
max_len = 64
batch_size = 32


## 학습 모델 로드
PATH = '/content/drive/MyDrive/Colab Notebooks/'
model = torch.load(PATH + 'KoBERT_감정분류.pt', map_location=torch.device('cpu'))  # 전체 모델을 통째로 불러옴, 클래스 선언 필수
model.load_state_dict(torch.load(PATH + 'model_state_dict.pt', map_location=torch.device('cpu')))  # state_dict를 불러 온 후, 모델에 저장

#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def new_softmax(a) : 
    c = np.max(a) # 최댓값
    exp_a = np.exp(a-c) # 각각의 원소에 최댓값을 뺀 값에 exp를 취한다. (이를 통해 overflow 방지)
    sum_exp_a = np.sum(exp_a)
    y = (exp_a / sum_exp_a) * 100
    return np.round(y, 3)


# 예측 모델 설정
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)

        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()
            # min_v = min(logits)
            # total = 0
            # probability = []
            # logits = np.round(new_softmax(logits), 3).tolist()
            # for logit in logits:
            #     # print(logit)
            #     probability.append(np.round(logit, 3))

            if np.argmax(logits) == 0:  emotion = "중립"
            elif np.argmax(logits) == 1: emotion = "분노"
            elif np.argmax(logits) == 2: emotion = '슬픔'
            elif np.argmax(logits) == 3: emotion = '행복'
            elif np.argmax(logits) == 4: emotion = '당황'
            elif np.argmax(logits) == 5: emotion = '불안'
            print('1')

            # probability.append(emotion)
            # print(probability)
    # return probability
    return emotion

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [84]:
device = torch.device("cpu")


In [100]:
predict("아.. 오늘 날씨 너무 구린데,,")

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


1


'슬픔'

In [48]:
import re
import os
import pandas as pd

In [46]:
def katalk_msg_parse(file_path):
  my_katalk_data = list()
  katalk_msg_pattern = "[0-9]{4}[년.] [0-9]{1,2}[일.] [0-9]{1,2}[오\S.] [0-9]{1,2}:[0-9]{1,2},.*:" #카카오톡 메시지 패턴
  date_info = "[0-9]{4}년 [0-9]{1,2}월 [0-9]{1,2}일 \S요일" #날짜 바뀌는 패턴
  in_out_info = "[0-9]{4}[년.] [0-9]{1,2}[월.] [0-9]{1,2}[일.] 오\S [0-9]{1,2}:[0-9]{1,2}:.*" #나갔다는 알림 무시
  money_text = "[0-9]{4}[년.] [0-9]{1,2}[일.] [0-9]{1,2}[오\S.] [0-9]{1,2}:[0-9]{1,2},.*:.[0-9]{1,3},.*원"#카카오페이 돈거래
  audio_visual_text = "^동영상$|^사진$|^사진 [0-9]{1,2}장$|^이모티콘$" #사진이나 동영상 메세지, 이모티콘은 추가 x

  for line in open(file_path):
    if re.match(date_info, line) or re.match(in_out_info, line) or re.match(money_text, line):
      continue
    elif line == '\n':
      continue
    elif re.match(katalk_msg_pattern, line):
      line = line.split(",") #,기준 2020. 1. 23. 11:57, 윤승환 : 너 친구약속 만나?
      date_time = line[0]
      user_text = line[1].split(" : ", maxsplit=1)
      user_name = user_text[0].strip()
      text = user_text[1].strip()
      if re.match(audio_visual_text, text): #사진 짤
        continue
      elif my_katalk_data and my_katalk_data[-1]['user_name'] == user_name: #동일 인물의 발화에 대해서..
        my_katalk_data[-1]['text'] += " "+ text
      else:
        my_katalk_data.append({"date_time": date_time,
                              "user_name": user_name,
                              "text": text})
    else:
      if len(my_katalk_data) > 0:
        my_katalk_data[-1]['text'] += "\n"+line.strip() #의도적으로 문장을 나눈 경우
    
  my_katalk_df = pd.DataFrame(my_katalk_data)

  return my_katalk_df


In [49]:
f_path = '/content/drive/MyDrive/Colab Notebooks'
f_name = 'katalk_data.txt'

df = katalk_msg_parse(os.path.join(f_path, f_name))
df[100:150]

,date_time,user_name,text
100,2020. 1. 24. 12:08,윤승환,감기그때 엄청심해짜나 ㅋㅋㅋㅋ
101,2020. 1. 24. 12:08,17김승주,그래 어쩐지 술때문이 아니라 감기때문이얐구나
102,2020. 1. 24. 12:08,윤승환,목소리 엄청 잠겼어 응응 어카지 나는 만약 너도 안간다하면 나도 안가고싶거든 왜냐면...
103,2020. 1. 24. 12:09,17김승주,ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 아니 술마시지말고 카페가 그럼
104,2020. 1. 24. 12:09,윤승환,아그러까 근데 승주도 와? ㅋㅋㅋㅋㅋㅋ
105,2020. 1. 24. 12:10,17김승주,나 근데 오늘 피부과가서 일욜에 너무 피부상태 안좋으면 안나갈것같은대
106,2020. 1. 24. 12:10,윤승환,그러면
107,2020. 1. 24. 12:10,17김승주,안나갈것같긴해
108,2020. 1. 24. 12:10,윤승환,쉬어 ㅋㅋㅋㅋㅋㅋㅋㅋ 나는 오늘 간만에 방청소할거야
109,2020. 1. 24. 12:11,17김승주,응응...나도 나가고는싶어 간만에가 얼마만?


In [103]:
for i in range(10):
  df['emotion'][i] = predict(df['text'][i])

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


1
1
1
1
1
1
1
1
1
1


In [105]:
df[:10]

,date_time,user_name,text,emotion
0,2020. 1. 22. 23:59,윤승환,봉투가 도착했어요.,불안
1,2020. 1. 23. 03:43,17김승주,우우 어우 너무 머리아파,슬픔
2,2020. 1. 23. 04:02,윤승환,안자?,중립
3,2020. 1. 23. 04:06,17김승주,자려고 눕긴했엉 자야지.. 승환 속 괜찮아?ㅋㅋㅋ,행복
4,2020. 1. 23. 11:57,윤승환,ㅋㅋㅋㅋㅋㅋ 와 너 친구약속 만나?,행복
5,2020. 1. 23. 11:58,17김승주,응응 약속때문에 9시에 일어났지ㅋㅋ,중립
6,2020. 1. 23. 12:00,윤승환,대단한걸 ㅋㅋㅋㅋㅋㅋ 나는 지금 일어나써,슬픔
7,2020. 1. 23. 12:00,17김승주,푹잤네ㅋㅋㅋㅋ 근데 머리가 아파 속은괜찮은데.. 넌 괜찮니,불안
8,2020. 1. 23. 12:01,윤승환,나 이제 완전 괜찮은데? ㅋㅋㅋㅋㅋㅋㅋㅋㅋ,당황
9,2020. 1. 23. 12:01,17김승주,대단해,중립


In [107]:
predict("나는 지금 일어나써")

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


1


'당황'